# 7. Vertical Vibration of Quarter Car Model

This notebook introduces the base excitation system by examning the behavior of a quarter car model.

After the completion of this assignment students will be able to:

- excite a system with a sinusoidal input
- understand the difference in transient and steady state solutions
- create a frequency response plot
- define resonance and determine the parameters that cause resonance

![](fig/07/quarter-car.jpg)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
from resonance.linear_systems import SimpleQuarterCarSystem

In [ ]:
sys = SimpleQuarterCarSystem()

The simple quarter car model has a suspension stiffness and damping, along with the sprung car mass in kilograms, and a travel speed parameter in meters per second.

In [ ]:
sys.constants

In [ ]:
sys.coordinates

In [ ]:
sys.speeds

# A sinusoidal road

The road is described as:

$$y(t) = Ysin\omega_b t$$

where $Y$ is the amplitude of the sinusoidal road undulations and $\omega_b$ is the frequency of the a function of the car's speed. If the distance between the peaks (amplitude 0.01 meters) of the sinusoidal road is 6 meters and the car is traveling at 7.5 m/s calculate what the frequency will be.

In [ ]:
Y = 0.01  # m
v = sys.constants['travel_speed']
bump_distance = 6  # m
wb = v / bump_distance * 2 * np.pi  # rad /s

Now with the amplitude and frequency set you can use the `sinusoidal_base_displacing_response()` function to simulate the system.

In [ ]:
traj = sys.sinusoidal_base_displacing_response(Y, wb, 20.0)
traj.head()

In [ ]:
traj.plot(subplots=True);

In [ ]:
sys.animate_configuration(interval=1000/100)

**Exercise**

Try different travel speeds and see what kind of behavior you can observe. Make sure to set the `travel_speed` constant and the frequency value for `sinusoidal_base_displacing_response()` to be consistent.

# Transmissibility

When designing a car the designer wants the riders to feel comfortable and to isolate them from the road's bumps. There are two important aspect to investigate. The first is called *displacement transmissibility* and is a ratio between the ampitude of the steady state motion and the ampitude of the sinusoidal base displacement. So in our case this would be:

$$ \frac{X}{Y}(\omega_b) = \frac{\textrm{Steady State Amplitude}}{\textrm{Base Displacement Amplitude}} $$

This can be plotted as a function of the base displacement frequency.

In [ ]:
from scipy.optimize import curve_fit
def cosine_func(times, amp, freq, phase_angle):
    return amp * np.cos(freq * times - phase_angle)
frequencies = np.linspace(1.0, 10.0, num=100)
   
amplitudes = []
    
for omega in frequencies:
    traj = sys.sinusoidal_base_displacing_response(Y, omega, 20.0)
    popt, pcov = curve_fit(cosine_func,
                           traj[10:].index, traj[10:].car_vertical_position,
                           p0=(Y, omega, 0.05))
    amplitudes.append(abs(popt[0]))

amplitudes = np.array(amplitudes)

fig, ax = plt.subplots(1, 1, sharex=True)
ax.set_xlabel('$\omega$ [rad/s]')
ax.set_ylabel('Displacement Transmisibility') 

ax.axvline(np.sqrt(sys.constants['suspension_stiffness'] / sys.constants['sprung_mass']), color='black')
ax.plot(frequencies, amplitudes / Y)
ax.grid();

The second thing to investigate is the *force transmissibility*. This is the ratio of the force applied by the suspension to the spurng car. Riders will feel this force when the car travels over bumps. Reducing this 

In [ ]:
def force_on_car(suspension_damping, suspension_stiffness, car_vertical_position, car_vertical_velocity, time):
    y = Y * np.sin(wb * time)
    yd = Y * wb * np.cos(wb * time)
    return (suspension_damping * (car_vertical_velocity - yd) +
            suspension_stiffness * (car_vertical_position - y))

In [ ]:
sys.add_measurement('force_on_car', force_on_car)

In [ ]:
traj = sys.sinusoidal_base_displacing_response(Y, omega, 20.0)
traj.plot(subplots=True)

In [ ]:
frequencies = np.linspace(1.0, 10.0, num=100)
   
amplitudes = []
    
for omega in frequencies:
    traj = sys.sinusoidal_base_displacing_response(Y, omega, 20.0)
    popt, pcov = curve_fit(cosine_func,
                           traj[10:].index, traj[10:].force_on_car,
                           p0=(sys.constants['suspension_stiffness'] * Y, omega, 0.01))
    amplitudes.append(abs(popt[0]))

amplitudes = np.array(amplitudes)

fig, ax = plt.subplots(1, 1, sharex=True)
ax.set_xlabel('$\omega$ [rad/s]')
ax.set_ylabel('Force Transmisibility') 

ax.axvline(np.sqrt(sys.constants['suspension_stiffness'] / sys.constants['sprung_mass']), color='black')
ax.plot(frequencies, amplitudes / sys.constants['suspension_stiffness'] / Y)
ax.grid();

#  Arbitrary Periodic Forcing (Fourier Series)

Fourier discovered that any periodic function with a period $T$ can be described by an infinite series of sums of sines and cosines. See the wikipedia article for more info (https://en.wikipedia.org/wiki/Fourier_series). The key equation is this:

$$ F(t) = \frac{a_0}{2} + \sum_{n=1}^\inf (a_n \cos n\omega_T t + b_n \sin n \omega_T t)$$

The terms $a_0, a_n, b_n$ are called the Fourier coefficients and are defined as such:

$$ a_0 = \frac{2}{T} \int_0^T F(t) dt$$

$$ a_n = \frac{2}{T} \int_0^T F(t) \cos n \omega_T t dt \quad \textrm{for} \quad n = 1, 2, \ldots $$

$$ b_n = \frac{2}{T} \int_0^T F(t) \sin n \omega_T t dt \quad \textrm{for} \quad n = 1, 2, \ldots $$


## Introduction to SymPy

SymPy is a Python package for symbolic computing. It can do many symbolic operations, for instance, integration, differentiation, linear algebra, etc. See http://sympy.org for more details of the features and the documentation. Today we will cover how to do integrals using SymPy and use it to find the Fourier series that represents a sawtooth function.

In [ ]:
import sympy as sm

The function `init_printing()` enables LaTeX based rendering in the Jupyter notebook of all SymPy objects.

In [ ]:
sm.init_printing()

Symbols can be created by using the `symbols()` function.

In [ ]:
x, y, z = sm.symbols('x, y, z')

In [ ]:
x, y, z

The `integrate()` function allows you to do symbolic indefinite or definite integrals. Note that the constants of integration are not included in indefinite integrals.

In [ ]:
sm.integrate(x * y, x)

The `Integral` class creates and unevaluated integral, where as the `integrate()` function automatically evaluates the integral.

In [ ]:
expr = sm.Integral(x * y, x)
expr

To evaluate the unevaluated form you call the `.doit()` method. Note that all unevaluated SymPy objects have this method.

In [ ]:
expr.doit()

This shows how to create an unevaluated definite integral, store it in a variable, and then evaluate it.

In [ ]:
expr = sm.Integral(x * y, (x, 0, 5))
expr

In [ ]:
expr.doit()

# Fourier Coefficients for the Sawtooth function

Now let's compute the Fourier coefficients for a saw tooth function. The function that describes the saw tooth is:

$$
F(t) = 
\begin{cases} 
      A \left( \frac{4t}{T} - 1 \right) & 0 \leq t \leq T/2 \\
      A \left( 3 - \frac{4t}{t} \right) & T/2 \leq t \leq T 
\end{cases}
$$

where:

- $A$ is the amplitude of the saw tooth
- $T$ is the period of the saw tooth
- $\omega_T$ is the frequency of the saw tooth, i.e. $\omega_T = \frac{2\pi}{T}$
- $t$ is time

This is a piecewise function with two parts from $t=0$ to $t=T$.

In [ ]:
A, T, wT, t = sm.symbols('A, T, omega_T, t', real=True, positive=True)
A, T, wT, t

The first Fourier coefficient $a_0$ describes the average value of the periodic function. and is:

$$a_0 = \frac{2}{T} \int_0^T F(t) dt$$

This integral will have to be done in two parts:

$$a_0 = a_{01} + a_{02} = \frac{2}{T} \int_0^{T/2} F(t) dt + \frac{2}{T} \int_{T/2}^T F(t) dt$$

These two integrals are evaluated below. Note that $a_0$ evaluates to zero. This is because the average of our function is 0.

In [ ]:
ao_1 = 2 / T * sm.Integral(A * (4 * t / T - 1), (t, 0, T / 2))
ao_1

In [ ]:
ao_1.doit()

In [ ]:
ao_2 = 2 / T * sm.Integral(A * (3 - 4 * t / T), (t, T / 2, T))
ao_2

In [ ]:
ao_2.doit()

But SymPy can also handle piecewise directly. The following shows how to define a piecewise function.

In [ ]:
F_1 = A * (4 * t / T - 1)
F_2 = A * (3 - 4 * t / T)

In [ ]:
F = sm.Piecewise((F_1, t<=T/2),
                 (F_2, T/2<t))
F

The integral can be taken of the entire piecewise function in one call.

In [ ]:
sm.integrate(F, (t, 0, T))

Now the Fourier coefficients $a_n$ and $b_n$ can be computed.

$$
a_n = \frac{2}{T}\int_0^T F(t) \cos n\omega_Tt dt \\
b_n = \frac{2}{T}\int_0^T F(t) \sin n\omega_Tt dt
$$

In [ ]:
n = sm.symbols('n', real=True, positive=True)

For $a_n$:

In [ ]:
an = 2 / T * sm.Integral(F * sm.cos(n * wT * t), (t, 0, T))
an

In [ ]:
an.doit()

This can be simplified:

In [ ]:
an = an.doit().simplify()
an

Now substitute the $2\pi/T$ for $\omega_T$.

In [ ]:
an = an.subs({wT: 2 * sm.pi / T})
an

Let's see how this function varies with increasing $n$. We will use a loop but the SymPy expressions will not automatically display because they are inside a loop. So we need to use SymPy's `latex()` function and the IPython display tools. SymPy's `latex()` function  transforms the SymPy expression into a string of matching LaTeX commands.

In [ ]:
sm.latex(an, mode='inline')

The `display()` and `LaTeX()` functions then turn the LaTeX string in to a displayed version.

In [ ]:
from IPython.display import display, Latex 

Now we can see how $a_n$ varies with $n=1,2,\ldots$.

In [ ]:
for n_i in range(1, 6):
    ans = an.subs({n: n_i})
    display(Latex('$a_{} = $'.format(n_i) + sm.latex(ans, mode='inline')))

For even $n$ values the coefficient is zero and for even values it variers with the inverse of $n^2$. More precisely:

$$
a_n =
\begin{cases}
0 & \textrm{if }n\textrm{ is even} \\
-\frac{8A}{n^2\pi^2} & \textrm{if }n\textrm{ is odd}
\end{cases}
$$

SymPy can actually reduce this further if your set the assumption that $n$ is an integer.

In [ ]:
n = sm.symbols('n', real=True, positive=True, integer=True)
an = 2 / T * sm.Integral(F * sm.cos(n * wT * t), (t, 0, T))
an = an.doit().simplify()
an.subs({wT: 2 * sm.pi / T})

The odd and even versions can be computed by setting the respective assumptions.

In [ ]:
n = sm.symbols('n', real=True, positive=True, integer=True, odd=True)
an = 2 / T * sm.Integral(F * sm.cos(n * wT * t), (t, 0, T))
an = an.doit().simplify()
an.subs({wT: 2 * sm.pi / T})

Note that $b_n$ is always zero:

In [ ]:
bn = 2 / T * sm.Integral(F * sm.sin(n * wT * t), (t, 0, T))
bn

In [ ]:
bn.doit().simplify().subs({wT: 2 * sm.pi / T})

# Numerical evalution of the Fourier Series

Now the Fourier coefficients can be used to plot the approximation of the saw tooth forcing function.

In [ ]:
import numpy as np

The following function plots the actual sawtooth function. It does it all in one line by cleverly using the absolute value and the modulo functions.

In [ ]:
def sawtooth(A, T, t):
    return (4 * A / T) * (T / 2 - np.abs(t % T - T / 2) ) - A

In [ ]:
A = 1
T = 2
t = np.linspace(0, 5, num=500)

In [ ]:
plt.figure()

plt.plot(t, sawtooth(A, T, t));

# Exercise

Write a function that computes the Fourier approximation of the sawtooth function for a given value of $n$, i.e. using a finite number of terms. Then plot it for $n=2, 4, 6, 8, 10$ on top of the actual sawtooth function. How many terms of the infinite series are needed to get a good sawtooth?

In [ ]:
def sawtooth_approximation(n, A, T, t):
    # odd values of indexing variable up to n
    n_i = np.arange(1, n+1, 2)[:, np.newaxis]
    # cos coefficients
    an = -8 * A / (n_i**2 * np.pi**2)
    # sawtooth frequency
    wT = 2 * np.pi / T
    # sum of n cos functions
    f = np.sum(an * np.cos(n_i * wT * t), axis=0)
    return f

# plot sawtooth
f = sawtooth(A, T, t)

fig, ax = plt.subplots()
ax.plot(t, f, color='k', label='true sawtooth')

for n in np.arange(2, 12, 2):
    f_approx = sawtooth_approximation(n, A, T, t)
    ax.plot(t, f_approx, label='n = {}'.format(n))

ax.legend()
# zoom in a bit on the interesting bit
ax.set_xlim(0, T)

# Apply the sawtooth to the quarter car

Now that you know the Fourier series coefficients. Calculate them for a suitable number of terms and simulate them with the `sys.periodic_base_displacing_response()` function.

In [ ]:
def fourier_coeffs(A, T, N):
    n = np.arange(1, N+1, 2)
    an_odd = -8 * A / (n**2 * np.pi**2)
    an_even = np.zeros_like(an_odd)
    an = np.zeros(an_odd.size + an_even.size)
    # zero-th element of an is actually a1
    an[::2] = an_odd
    an[1::2] = an_even
    return 0, an, np.zeros_like(an)

a0, an, bn = fourier_coeffs(0.01, 2 * np.pi / wb, 100)

traj = sys.periodic_base_displacing_response(a0, an, bn, wb, 20.0)

In [ ]:
traj.plot(subplots=True)